# Repartitioning DataFrames

Partitions are a central concept in Apache Spark. They are used for distributing and parallelizing work onto different executors, which run on multiple servers. 

### Determining Partitions
Basically Spark uses two different strategies for splitting up data into multiple partitions:
1. When Spark loads data, the records are put into partitions along natural borders. For example every HDFS block (and thereby every file) is represented by a different partition. Therefore the number of partitions of a DataFrame read from disk is solely determined by the number of HDFS blocks
2. Certain operations like `JOIN`s and aggregations require that records with the same key are physically in the same partition. This is achieved by a shuffle phase. The number of partitions is specified by the global Spark configuration variable `spark.sql.shuffle.partitions` which has a default value of 200.

### Repartitiong Data
Since partitions have a huge influence on the execution, Spark also allows you to explicitly change the partitioning schema of a DataFrame. This makes sense only in a very limited (but still important) set of cases, which we will discuss in this notebook.

### Weather Example
Surprise, surprise, we will again use the weather example and see what explicit repartitioning gives us.

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

if not 'spark' in locals():
    spark = SparkSession.builder \
        .master("local[*]") \
        .config("spark.driver.memory","24G") \
        .getOrCreate()

spark

/opt/anaconda3/lib/python3.10/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/25 17:03:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/25 17:03:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Disable Automatic Broadcast JOINs
In order to see the shuffle operations, we need to prevent Spark from executiong `JOIN` operations as broadcast joins. Again this can be turned off by setting the Spark configuration variable `spark.sql.autoBroadcastJoinThreshold` to -1.

In [2]:
spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

# 1 Load Data

First we load the weather data, which consists of the measurement data and some station metadata.

In [3]:
storageLocation = "s3://dimajix-training/data/weather"
# storageLocation = "/dimajix/data/weather-noaa-sample"

## 1.1 Load Measurements

Measurements are stored in multiple directories (one per year). But we will limit ourselves to a single year in the analysis to improve readability of execution plans.

In [4]:
from functools import reduce

# Read in all years, store them in an Python array
raw_weather_per_year = [spark.read.text(storageLocation + "/" + str(i)).withColumn("year", f.lit(i)) for i in range(2003,2015)]

# Union all years together
raw_weather = reduce(lambda l,r: l.union(r), raw_weather_per_year)                        

Use a single year to keep execution plans small

In [5]:
raw_weather = spark.read.text(storageLocation + "/2003").withColumn("year", f.lit(2003))

### Extract Measurements

Measurements were stored in a proprietary text based format, with some values at fixed positions. We need to extract these values with a simple `SELECT` statement.

In [6]:
weather = raw_weather.select(
    f.col("year"),
    f.substring(f.col("value"),5,6).alias("usaf"),
    f.substring(f.col("value"),11,5).alias("wban"),
    f.substring(f.col("value"),16,8).alias("date"),
    f.substring(f.col("value"),24,4).alias("time"),
    f.substring(f.col("value"),42,5).alias("report_type"),
    f.substring(f.col("value"),61,3).alias("wind_direction"),
    f.substring(f.col("value"),64,1).alias("wind_direction_qual"),
    f.substring(f.col("value"),65,1).alias("wind_observation"),
    (f.substring(f.col("value"),66,4).cast("float") / f.lit(10.0)).alias("wind_speed"),
    f.substring(f.col("value"),70,1).alias("wind_speed_qual"),
    (f.substring(f.col("value"),88,5).cast("float") / f.lit(10.0)).alias("air_temperature"),
    f.substring(f.col("value"),93,1).alias("air_temperature_qual")
)

## 1.2 Load Station Metadata

We also need to load the weather station meta data containing information about the geo location, country etc of individual weather stations.

In [7]:
stations = spark.read \
    .option("header", True) \
    .csv(storageLocation + "/isd-history")

# 2 Partitions

Since partitions is a concept at the RDD level and a DataFrame per se does not contain an RDD, we need to access the RDD in order to inspect the number of partitions.

In [8]:
weather.rdd.getNumPartitions()

37

## 2.1 Repartitioning Data

You can repartition any DataFrame by specifying the target number of partitions and the partitioning columns. While it should be clear what *number of partitions* actually means, the term *partitionng columns* might require some explanation.

### Partitioning Columns
Except for the case when Spark initially reads data, all DataFrames are partitioned along *partitioning columns*, which means that all records having the same values in the corresponding columns will end up in the same partition. Spark implicitly performs such repartitioning as shuffle operations for `JOIN`s and grouped aggregation (except when a DataFrame already has the correct partitioning columns and number of partitions)

### Manual Repartitioning
As already mentioned, you can explicitly repartition a DataFrame using teh `repartition()` method.

In [10]:
result = weather.repartition(10, weather["usaf"], weather["wban"])
result.rdd.getNumPartitions()

10

### Effect of Repartition

Apart from introducing an additional shuffle operation, repartitioning a dataset will effectevely control the level of parallelism

In [11]:
result = weather.repartition(20).select(f.count("*"))
result.toPandas()

,count(1)
0,1807253


In [12]:
result.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[count(1)])
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=86]
   +- *(1) HashAggregate(keys=[], functions=[partial_count(1)])
      +- Exchange RoundRobinPartitioning(20), REPARTITION_BY_NUM, [plan_id=78]
         +- FileScan text [] Batched: false, DataFilters: [], Format: Text, Location: InMemoryFileIndex(1 paths)[file:/dimajix/data/weather-noaa-sample/2003], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<>




# 3 Repartition & Joins

As already mentioned, Spark implicitly performs a repartitioning aka shuffle for `JOIN` operations.

### Execution Plan

So let us inspect the execution plan of a `JOIN` operation.

In [13]:
result = weather.join(stations, ["usaf", "wban"])
result.explain()

== Physical Plan ==
*(5) Project [usaf#100, wban#101, 2003 AS year#96, date#102, time#103, report_type#104, wind_direction#105, wind_direction_qual#106, wind_observation#107, wind_speed#108, wind_speed_qual#109, air_temperature#110, air_temperature_qual#111, STATION NAME#144, CTRY#145, STATE#146, ICAO#147, LAT#148, LON#149, ELEV(M)#150, BEGIN#151, END#152]
+- *(5) SortMergeJoin [usaf#100, wban#101], [USAF#142, WBAN#143], Inner
   :- *(2) Sort [usaf#100 ASC NULLS FIRST, wban#101 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(usaf#100, wban#101, 200), ENSURE_REQUIREMENTS, [plan_id=132]
   :     +- *(1) Project [substring(value#94, 5, 6) AS usaf#100, substring(value#94, 11, 5) AS wban#101, substring(value#94, 16, 8) AS date#102, substring(value#94, 24, 4) AS time#103, substring(value#94, 42, 5) AS report_type#104, substring(value#94, 61, 3) AS wind_direction#105, substring(value#94, 64, 1) AS wind_direction_qual#106, substring(value#94, 65, 1) AS wind_observation#107, (cast

### Remarks

As we already discussed, each `JOIN` is executed with the following steps
1. Filter `NULL` values (it's an inner join)
2. Repartition DataFrame on the join columns with 200 partitions
3. Sort each partition independently
4. Perform a `SortMergeJoin`

## 3.1 Pre-partition data (first try)

Now let us try what happens when we explicitly repartition the data before the join operation.

In [15]:
weather_rep = weather.repartition(10, weather["usaf"], weather["wban"])
weather_rep.rdd.getNumPartitions()

10

In [16]:
stations_rep = stations.repartition(10, stations["usaf"], stations["wban"])
stations_rep.rdd.getNumPartitions()

10

#### Execution Plan

Let's analyze the resulting execution plan. Ideally all the preparation work before the `SortMergeJoin` happens before the `cache` operation.

In [17]:
result = weather_rep.join(stations_rep, ["usaf","wban"])
result.explain()

== Physical Plan ==
*(5) Project [usaf#100, wban#101, 2003 AS year#96, date#102, time#103, report_type#104, wind_direction#105, wind_direction_qual#106, wind_observation#107, wind_speed#108, wind_speed_qual#109, air_temperature#110, air_temperature_qual#111, STATION NAME#144, CTRY#145, STATE#146, ICAO#147, LAT#148, LON#149, ELEV(M)#150, BEGIN#151, END#152]
+- *(5) SortMergeJoin [usaf#100, wban#101], [USAF#142, WBAN#143], Inner
   :- *(2) Sort [usaf#100 ASC NULLS FIRST, wban#101 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(usaf#100, wban#101, 200), REPARTITION_BY_NUM, [plan_id=239]
   :     +- *(1) Project [substring(value#94, 5, 6) AS usaf#100, substring(value#94, 11, 5) AS wban#101, substring(value#94, 16, 8) AS date#102, substring(value#94, 24, 4) AS time#103, substring(value#94, 42, 5) AS report_type#104, substring(value#94, 61, 3) AS wind_direction#105, substring(value#94, 64, 1) AS wind_direction_qual#106, substring(value#94, 65, 1) AS wind_observation#107, (cast(

### Observations

Spark removed our explicit repartition, since it doesn't help and replaced it with the implicit repartition with 200 partitions

## 3.2 Pre-partition and Cache (second try)

Now let us try if we can cache the shuffle (repartition) and sort operation. This is useful in cases, where you have to perform multiple joins on the same set of columns, for example with different DataFrames.

So let's simply repartition the `weather` DataFrame on the two columns `usaf` and `wban`.

In [18]:
weather_rep = weather.repartition(20, weather["usaf"], weather["wban"])
weather_rep.cache()

DataFrame[year: int, usaf: string, wban: string, date: string, time: string, report_type: string, wind_direction: string, wind_direction_qual: string, wind_observation: string, wind_speed: double, wind_speed_qual: string, air_temperature: double, air_temperature_qual: string]

#### Execution Plan

Let's analyze the resulting execution plan. Ideally all the preparation work before the `SortMergeJoin` happens before the `cache` operation.

In [19]:
result = weather_rep.join(stations, ["usaf","wban"])
result.explain()

== Physical Plan ==
*(4) Project [usaf#100, wban#101, year#96, date#102, time#103, report_type#104, wind_direction#105, wind_direction_qual#106, wind_observation#107, wind_speed#108, wind_speed_qual#109, air_temperature#110, air_temperature_qual#111, STATION NAME#144, CTRY#145, STATE#146, ICAO#147, LAT#148, LON#149, ELEV(M)#150, BEGIN#151, END#152]
+- *(4) SortMergeJoin [usaf#100, wban#101], [USAF#142, WBAN#143], Inner
   :- *(1) Sort [usaf#100 ASC NULLS FIRST, wban#101 ASC NULLS FIRST], false, 0
   :  +- *(1) Filter (isnotnull(usaf#100) AND isnotnull(wban#101))
   :     +- InMemoryTableScan [year#96, usaf#100, wban#101, date#102, time#103, report_type#104, wind_direction#105, wind_direction_qual#106, wind_observation#107, wind_speed#108, wind_speed_qual#109, air_temperature#110, air_temperature_qual#111], [isnotnull(usaf#100), isnotnull(wban#101)]
   :           +- InMemoryRelation [year#96, usaf#100, wban#101, date#102, time#103, report_type#104, wind_direction#105, wind_direction_qu

### Remarks

Caching seems to fix the new partitioning, and the second DataFrame (`stations`) will be repartitioned accordingly.

## 3.3 Pre-partition and Cache (third try)

We already partially achieved our goal of caching all preparational work of the `SortMergeJoin`, but the sorting was still preformed after the caching. So let's try to insert an appropriate sort operation.

In [20]:
# Release cache to simplify execution plan
weather_rep.unpersist()

weather_rep = weather.repartition(200, weather["usaf"], weather["wban"]) \
    .orderBy(weather["usaf"], weather["wban"])
weather_rep.cache()

DataFrame[year: int, usaf: string, wban: string, date: string, time: string, report_type: string, wind_direction: string, wind_direction_qual: string, wind_observation: string, wind_speed: double, wind_speed_qual: string, air_temperature: double, air_temperature_qual: string]

#### Execution Plan

In [21]:
result = weather_rep.join(stations, ["usaf","wban"])
result.explain()

== Physical Plan ==
*(5) Project [usaf#100, wban#101, year#96, date#102, time#103, report_type#104, wind_direction#105, wind_direction_qual#106, wind_observation#107, wind_speed#108, wind_speed_qual#109, air_temperature#110, air_temperature_qual#111, STATION NAME#144, CTRY#145, STATE#146, ICAO#147, LAT#148, LON#149, ELEV(M)#150, BEGIN#151, END#152]
+- *(5) SortMergeJoin [usaf#100, wban#101], [USAF#142, WBAN#143], Inner
   :- *(2) Sort [usaf#100 ASC NULLS FIRST, wban#101 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(usaf#100, wban#101, 200), ENSURE_REQUIREMENTS, [plan_id=409]
   :     +- *(1) Filter (isnotnull(usaf#100) AND isnotnull(wban#101))
   :        +- InMemoryTableScan [year#96, usaf#100, wban#101, date#102, time#103, report_type#104, wind_direction#105, wind_direction_qual#106, wind_observation#107, wind_speed#108, wind_speed_qual#109, air_temperature#110, air_temperature_qual#111], [isnotnull(usaf#100), isnotnull(wban#101)]
   :              +- InMemoryRelation

#### Remarks

We actually created a worse situation: Now we have two sort operations! Definately not what we wanted to have.

So let's think for a moment: The `SortMergeJoin` requires that each partition is sorted, but after the repartioning occured. The `orderBy` operation we used above will create a global order over all partitions (and thereby destroy all the repartition work immediately). So we need something else, which still keeps the current partitions but only sort in each partition independently.

## 3.4 Pre-partition and Cache (final try)

Fortunately Spark provides a `sortWithinPartitions` method, which does exactly what it sounds like.

In [22]:
# Release cache to simplify execution plan
weather_rep.unpersist()

weather_rep = weather.repartition(200, weather["usaf"], weather["wban"]) \
    .sortWithinPartitions(weather["usaf"], weather["wban"])
weather_rep.cache()

DataFrame[year: int, usaf: string, wban: string, date: string, time: string, report_type: string, wind_direction: string, wind_direction_qual: string, wind_observation: string, wind_speed: double, wind_speed_qual: string, air_temperature: double, air_temperature_qual: string]

#### Execution Plan

In [23]:
result = weather_rep.join(stations, (weather["usaf"] == stations["usaf"]) & (weather["wban"] == stations["wban"]))
result.explain()

== Physical Plan ==
*(4) SortMergeJoin [usaf#100, wban#101], [usaf#142, wban#143], Inner
:- *(1) Filter (isnotnull(usaf#100) AND isnotnull(wban#101))
:  +- InMemoryTableScan [year#96, usaf#100, wban#101, date#102, time#103, report_type#104, wind_direction#105, wind_direction_qual#106, wind_observation#107, wind_speed#108, wind_speed_qual#109, air_temperature#110, air_temperature_qual#111], [isnotnull(usaf#100), isnotnull(wban#101)]
:        +- InMemoryRelation [year#96, usaf#100, wban#101, date#102, time#103, report_type#104, wind_direction#105, wind_direction_qual#106, wind_observation#107, wind_speed#108, wind_speed_qual#109, air_temperature#110, air_temperature_qual#111], StorageLevel(disk, memory, deserialized, 1 replicas)
:              +- *(2) Sort [usaf#100 ASC NULLS FIRST, wban#101 ASC NULLS FIRST], false, 0
:                 +- Exchange hashpartitioning(usaf#100, wban#101, 200), REPARTITION_BY_NUM, [plan_id=455]
:                    +- *(1) Project [2003 AS year#96, substring(

### Remarks

That looks really good. The filter operation is still executed after the cache, but that cannot be cached such that Spark uses this information.

So whenever you want to prepartition data, you need to execute the following steps:
* repartition with the join columns and default number of partitions
* sortWithinPartitions with the join columns
* probably cache (otherwise there is no benefit at all)

### Inspect WebUI

We can also inspect the WebUI and see how everything is executed.

Phase 1: Build cache

In [24]:
result.count()

1807253

Phase 2: Use cache

In [25]:
result.count()

1807253

# 4 Repartition & Aggregations

Similar to `JOIN` operations, Spark also requires an appropriate partitioning in grouped aggregations. Again, we can use the same strategy and appropriateky prepartition data in cases where multiple joins and aggregations are performed using the same columns.

## 4.1 Simple Aggregation

So let's perform the usual aggregation (but this time without a previous `JOIN`) with groups defined by the station id (`usaf` and `wban`).

In [26]:
result = weather.groupBy(weather["usaf"], weather["wban"]).agg(
        f.min(f.when(weather.air_temperature_qual == f.lit(1), weather.air_temperature)).alias('min_temp'),
        f.max(f.when(weather.air_temperature_qual == f.lit(1), weather.air_temperature)).alias('max_temp'),
)
result.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[usaf#100, wban#101], functions=[min(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END), max(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END)])
+- Exchange hashpartitioning(usaf#100, wban#101, 200), ENSURE_REQUIREMENTS, [plan_id=699]
   +- *(1) HashAggregate(keys=[usaf#100, wban#101], functions=[partial_min(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END), partial_max(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END)])
      +- *(1) Project [substring(value#94, 5, 6) AS usaf#100, substring(value#94, 11, 5) AS wban#101, (cast(cast(substring(value#94, 88, 5) as float) as double) / 10.0) AS air_temperature#110, substring(value#94, 93, 1) AS air_temperature_qual#111]
         +- FileScan text [value#94] Batched: false, DataFilters: [], Format: Text, Location: InMemoryFileIndex(1 paths)[file:/dimajix/data/weather

### Remarks
Each grouped aggregation is executed with the following steps:
1. Perform partial aggregation (`HashAggregate`)
2. Shuffle intermediate result (`Exchange hashpartitioning`)
3. Perform final aggregation (`HashAggregate`)

## 4.2 Aggregation after repartition

Now let us perform the same aggregation, but this time let's use the preaggregated weather data set `weather_rep` instead.

In [27]:
weather_rep = weather.repartition(87, weather["usaf"], weather["wban"])
weather_rep.unpersist()

DataFrame[year: int, usaf: string, wban: string, date: string, time: string, report_type: string, wind_direction: string, wind_direction_qual: string, wind_observation: string, wind_speed: double, wind_speed_qual: string, air_temperature: double, air_temperature_qual: string]

In [28]:
result = weather_rep.groupBy(weather["usaf"], weather["wban"]).agg(
        f.min(f.when(weather_rep.air_temperature_qual == f.lit(1), weather_rep.air_temperature)).alias('min_temp'),
        f.max(f.when(weather_rep.air_temperature_qual == f.lit(1), weather_rep.air_temperature)).alias('max_temp'),
)
result.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[usaf#100, wban#101], functions=[min(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END), max(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END)])
+- *(2) HashAggregate(keys=[usaf#100, wban#101], functions=[partial_min(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END), partial_max(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END)])
   +- Exchange hashpartitioning(usaf#100, wban#101, 87), REPARTITION_BY_NUM, [plan_id=731]
      +- *(1) Project [substring(value#94, 5, 6) AS usaf#100, substring(value#94, 11, 5) AS wban#101, (cast(cast(substring(value#94, 88, 5) as float) as double) / 10.0) AS air_temperature#110, substring(value#94, 93, 1) AS air_temperature_qual#111]
         +- FileScan text [value#94] Batched: false, DataFilters: [], Format: Text, Location: InMemoryFileIndex(1 paths)[file:/dimajix/data/weather-n

### Remarks
Spark obviously detects the correct partitioning of the `weather_rep` DataFrame. The sorting actually is not required, but does not hurt either (except performance...). Therefore only two steps are executed after the cache operation:
1. Partial aggregation (`HashAggregate`)
2. Final aggregation (`HashAggregate`)

But note that although you saved a shuffle operation of partial aggregates, in most cases it is not adviseable to prepartition data only for aggregations for the following reasons:
* You could perform all aggregations in a single `groupBy` and `agg` chain
* In most cases the preaggregated data is significantly smaller than the original data, therefore the shuffle doesn't hurt that much

# 5 Interaction between Join, Aggregate & Repartition

Now we have seen two operations which require a shuffle of the data. Of course Spark is clever enough to avoid an additional shuffle operation in chains of `JOIN` and grouped aggregations, which use the same aggregation columns.

## 5.1 Aggregation after Join on same key

So let's see what happens with a grouped aggregation after a join operation.

In [29]:
joined = weather.join(stations, (weather["usaf"] == stations["usaf"]) & (weather["wban"] == stations["wban"]))
result = joined.groupBy(weather["usaf"], weather["wban"]).agg(
        f.min(f.when(joined.air_temperature_qual == f.lit(1), joined.air_temperature)).alias('min_temp'),
        f.max(f.when(joined.air_temperature_qual == f.lit(1), joined.air_temperature)).alias('max_temp'),
)
result.explain()

== Physical Plan ==
*(5) HashAggregate(keys=[usaf#100, wban#101], functions=[min(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END), max(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END)])
+- *(5) HashAggregate(keys=[usaf#100, wban#101], functions=[partial_min(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END), partial_max(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END)])
   +- *(5) Project [usaf#100, wban#101, air_temperature#110, air_temperature_qual#111]
      +- *(5) SortMergeJoin [usaf#100, wban#101], [usaf#142, wban#143], Inner
         :- *(2) Sort [usaf#100 ASC NULLS FIRST, wban#101 ASC NULLS FIRST], false, 0
         :  +- Exchange hashpartitioning(usaf#100, wban#101, 200), ENSURE_REQUIREMENTS, [plan_id=790]
         :     +- *(1) Project [substring(value#94, 5, 6) AS usaf#100, substring(value#94, 11, 5) AS wban#101, (cast(cast(substring(value

### Remarks

As you can see, Spark performs a single shuffle operation. The order of operation is as follows:
1. Filter `NULL` values (it's an inner join)
2. Shuffle data on `usaf` and `wban`
3. Sort partitions by `usaf` and `wban`
4. Perform `SortMergeJoin`
5. Perform partial aggregation `HashAggregate`
6. Perform final aggregation `HashAggregate`

## 5.2 Aggregation after Join using repartitioned data

Of course we can also use the pre-repartitioned weather DataFrame. This will work as expected, Spark does not add any additional shuffle operation.

In [30]:
weather_rep = weather.repartition(84, weather["usaf"], weather["wban"])

joined = weather_rep.join(stations, ["usaf","wban"])
result = joined.groupBy(weather["usaf"], weather["wban"]).agg(
        f.min(f.when(joined.air_temperature_qual == f.lit(1), joined.air_temperature)).alias('min_temp'),
        f.max(f.when(joined.air_temperature_qual == f.lit(1), joined.air_temperature)).alias('max_temp'),
)
result.explain()

== Physical Plan ==
*(5) HashAggregate(keys=[usaf#100, wban#101], functions=[min(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END), max(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END)])
+- *(5) HashAggregate(keys=[usaf#100, wban#101], functions=[partial_min(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END), partial_max(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END)])
   +- *(5) Project [usaf#100, wban#101, air_temperature#110, air_temperature_qual#111]
      +- *(5) SortMergeJoin [usaf#100, wban#101], [USAF#142, WBAN#143], Inner
         :- *(2) Sort [usaf#100 ASC NULLS FIRST, wban#101 ASC NULLS FIRST], false, 0
         :  +- Exchange hashpartitioning(usaf#100, wban#101, 200), REPARTITION_BY_NUM, [plan_id=875]
         :     +- *(1) Project [substring(value#94, 5, 6) AS usaf#100, substring(value#94, 11, 5) AS wban#101, (cast(cast(substring(value#

Note that the explicit repartition has been removed by Spark - therefore it doesn't make any sense to `repartition` before a join operation.

## 5.3 Aggregation after Join with different key

So far we only looked at join and grouping operations using the same keys. If we use different keys (for example the country) in both operations, we expect Spark to add an additional shuffle operations. Let's see...

In [31]:
joined = weather.join(stations, ["usaf","wban"])
result = joined.groupBy(stations["ctry"]).agg(
        f.min(f.when(joined.air_temperature_qual == f.lit(1), joined.air_temperature)).alias('min_temp'),
        f.max(f.when(joined.air_temperature_qual == f.lit(1), joined.air_temperature)).alias('max_temp'),
)
result.explain()

== Physical Plan ==
*(6) HashAggregate(keys=[ctry#145], functions=[min(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END), max(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END)])
+- Exchange hashpartitioning(ctry#145, 200), ENSURE_REQUIREMENTS, [plan_id=975]
   +- *(5) HashAggregate(keys=[ctry#145], functions=[partial_min(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END), partial_max(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END)])
      +- *(5) Project [air_temperature#110, air_temperature_qual#111, CTRY#145]
         +- *(5) SortMergeJoin [usaf#100, wban#101], [USAF#142, WBAN#143], Inner
            :- *(2) Sort [usaf#100 ASC NULLS FIRST, wban#101 ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(usaf#100, wban#101, 200), ENSURE_REQUIREMENTS, [plan_id=958]
            :     +- *(1) Project [substring(value#94, 5, 6) AS usaf#100

## 5.4 Aggregation after Broadcast-Join 

If we use a broadcast join instead of a sort merge join, the we will have a shuffle operation for the aggregation again (since the broadcast join just avoids the shuffle). Let's verify that theory...

In [32]:
joined = weather.join(f.broadcast(stations), ["usaf","wban"])
result = joined.groupBy(weather["usaf"], weather["wban"]).agg(
        f.min(f.when(joined.air_temperature_qual == f.lit(1), joined.air_temperature)).alias('min_temp'),
        f.max(f.when(joined.air_temperature_qual == f.lit(1), joined.air_temperature)).alias('max_temp'),
)
result.explain()

== Physical Plan ==
*(3) HashAggregate(keys=[usaf#100, wban#101], functions=[min(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END), max(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END)])
+- Exchange hashpartitioning(usaf#100, wban#101, 200), ENSURE_REQUIREMENTS, [plan_id=1067]
   +- *(2) HashAggregate(keys=[usaf#100, wban#101], functions=[partial_min(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END), partial_max(CASE WHEN (cast(air_temperature_qual#111 as int) = 1) THEN air_temperature#110 END)])
      +- *(2) Project [usaf#100, wban#101, air_temperature#110, air_temperature_qual#111]
         +- *(2) BroadcastHashJoin [usaf#100, wban#101], [USAF#142, WBAN#143], Inner, BuildRight, false
            :- *(2) Project [substring(value#94, 5, 6) AS usaf#100, substring(value#94, 11, 5) AS wban#101, (cast(cast(substring(value#94, 88, 5) as float) as double) / 10.0) AS air_temperature#110, su

# 6 Coalesce

There is another use case for changing the number of partitions: Writing results to HDFS/S3/whatever. Per design Spark writes each partition into a separate file, and there is no way around that. But when partitions do not contain many records, this may not only be ugly, but also unperformant and might cause additional trouble. Specifically currently HDFS is not designed to handle many small files, but prefers fewer large files instead.

Therefore it is often desireable to reduce the number of partitions of a DataFrame just before writing the result to disk. You could perform this task by a `repartition` operation, but this is an expensive operation requiring an additional shuffle operation. Therefore Spark provides an additional method called `coalesce` which can be used to reduce the number of partitions without incurring an additional shuffle. Spark simply logically concatenates multiple partitions into new partitions.

### Inspect Number of Partitions

For this example, we will use the `weather_rep` DataFrame, which contains exactly 200 partitions.

In [34]:
weather_rep = weather.repartition(200, weather["usaf"], weather["wban"])
weather_rep.cache()

DataFrame[year: int, usaf: string, wban: string, date: string, time: string, report_type: string, wind_direction: string, wind_direction_qual: string, wind_observation: string, wind_speed: double, wind_speed_qual: string, air_temperature: double, air_temperature_qual: string]

In [35]:
weather_rep.rdd.getNumPartitions()

200

## 6.1 Merge Partitions using coalesce

In order to reduce the number of partitions, we simply use the `coalesce` method.

In [41]:
result = weather.join(stations, ["usaf", "wban"]).select(f.count("*"))
result.toPandas()
result.explain()

== Physical Plan ==
*(6) HashAggregate(keys=[], functions=[count(1)])
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=1687]
   +- *(5) HashAggregate(keys=[], functions=[partial_count(1)])
      +- *(5) Project
         +- *(5) SortMergeJoin [usaf#100, wban#101], [USAF#142, WBAN#143], Inner
            :- *(2) Sort [usaf#100 ASC NULLS FIRST, wban#101 ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(usaf#100, wban#101, 200), ENSURE_REQUIREMENTS, [plan_id=1670]
            :     +- *(1) Project [substring(value#94, 5, 6) AS usaf#100, substring(value#94, 11, 5) AS wban#101]
            :        +- *(1) Filter (isnotnull(substring(value#94, 5, 6)) AND isnotnull(substring(value#94, 11, 5)))
            :           +- FileScan text [value#94] Batched: false, DataFilters: [isnotnull(substring(value#94, 5, 6)), isnotnull(substring(value#94, 11, 5))], Format: Text, Location: InMemoryFileIndex(1 paths)[file:/dimajix/data/weather-noaa-sample/2003], PartitionFilters

In [42]:
result = weather.join(stations, ["usaf", "wban"]).coalesce(16).select(f.count("*"))
result.toPandas()
result.explain()

== Physical Plan ==
*(7) HashAggregate(keys=[], functions=[count(1)])
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=1799]
   +- *(6) HashAggregate(keys=[], functions=[partial_count(1)])
      +- Coalesce 16
         +- *(5) Project
            +- *(5) SortMergeJoin [usaf#100, wban#101], [USAF#142, WBAN#143], Inner
               :- *(2) Sort [usaf#100 ASC NULLS FIRST, wban#101 ASC NULLS FIRST], false, 0
               :  +- Exchange hashpartitioning(usaf#100, wban#101, 200), ENSURE_REQUIREMENTS, [plan_id=1779]
               :     +- *(1) Project [substring(value#94, 5, 6) AS usaf#100, substring(value#94, 11, 5) AS wban#101]
               :        +- *(1) Filter (isnotnull(substring(value#94, 5, 6)) AND isnotnull(substring(value#94, 11, 5)))
               :           +- FileScan text [value#94] Batched: false, DataFilters: [isnotnull(substring(value#94, 5, 6)), isnotnull(substring(value#94, 11, 5))], Format: Text, Location: InMemoryFileIndex(1 paths)[file:/dimajix/data/w

## 6.2 Saving files

We already discussed that Spark writes a separate file per partition. So let's see the result when we write the `weather` DataFrame with 20 partitions. We will comapre the results of `repartition` and `coalesce`.

### Write 20 Partitions with `repartition`

In [43]:
weather.repartition(20).write.mode("overwrite").parquet("/tmp/weather_rep")

#### Inspect the Result
Using a simple HDFS CLI util, we can inspect the result on HDFS.

In [ ]:
%%bash
hdfs dfs -ls /tmp/weather_rep

### Write 20 Partitions with `coalesce`

Now let's write the `coalesce`d DataFrame and inspect the result on HDFS

In [45]:
weather.coalesce(20).write.mode("overwrite").parquet("/tmp/weather_rep")

#### Inspect Result

In [ ]:
%%bash
hdfs dfs -ls /tmp/weather_small